In [28]:
import os
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
from datetime import date

In [29]:
channel_id = 'UCsqjHFMB_JYTaEnf_vmTNqg'
api_key = os.environ['YOUTUBE_API_KEY']

# print(api_key)

# Extract

##### Extract channel stats

In [30]:
def channel_stats(api_key, channel_id):
    channel_stats_url = f"https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2Cstatistics&id={channel_id}&key={api_key}"
    # print(channel_stats_url)
    response = requests.get(channel_stats_url)
    channel_data = json.loads(response.text)
    # print(channel_data)
    return channel_data

##### Extract channel videos

In [31]:
def get_channel_videos(api_key,channel_id):
    page_count = 0
    url = f"https://youtube.googleapis.com/youtube/v3/search?channelId={channel_id}&maxResults=50&order=date&type=video&key={api_key}"
    
    
    def get_total_results(api_key,channel_id):
        response = requests.get(url)
        data = json.loads(response.text)
        totalResults = data['pageInfo']['totalResults']
        
        return totalResults
    
    

    def get_channel_videos_per_page(url):
        channel_videos = []
    
        response = requests.get(url)
        data = json.loads(response.text)
        nextPageToken = data.get('nextPageToken')
        for i in range(len(data['items'])):
            videoId = data['items'][i]['id']['videoId']
            channel_videos.append(videoId)

        return nextPageToken, channel_videos

    nextPageToken, channel_videos = get_channel_videos_per_page(url)
    totalResults = get_total_results(api_key, channel_id)
    while nextPageToken and (page_count < (totalResults/50)):
        next_page_url = f"{url}&pageToken={nextPageToken}"
        nextPageToken, new_channel_videos = get_channel_videos_per_page(next_page_url)
        channel_videos.extend(new_channel_videos)
        page_count += 1

    # print(channel_videos)
    return channel_videos

# get_channel_videos(api_key, channel_id)


##### Extract video stats

In [32]:

def get_video_stats(api_key):
    videos_stats = []
    videoIds = get_channel_videos(api_key, channel_id)

    for videoId in videoIds:
        video_data= {}
        video_stats_url = f"https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2Cstatistics&id={videoId}&key={api_key}"
        response = requests.get(video_stats_url)
        data = json.loads(response.text)

        video_data = {
            'publishedAt' : data['items'][0]['snippet']['publishedAt'],
            'title' : data['items'][0]['snippet']['title'],
            'views' : data['items'][0]['statistics']['viewCount'],
            'likes' : data['items'][0]['statistics']['likeCount'],
            'comments' : data['items'][0]['statistics']['commentCount'],

        }
        videos_stats.append(video_data)


    return videos_stats

In [33]:
channel_data = channel_stats(api_key,channel_id)

In [34]:
channel_data

{'kind': 'youtube#channelListResponse',
 'etag': 'nohf55NVWgInmoyacQXr2IQGOh8',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '42jg5aGyKmSRFsDD-IpEECkkfgU',
   'id': 'UCsqjHFMB_JYTaEnf_vmTNqg',
   'snippet': {'title': 'Doug DeMuro',
    'description': '',
    'customUrl': 'dougdemuro',
    'publishedAt': '2013-09-11T19:26:56Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AMLnZu95bYnhPmv4vYZz9QYf-6c6uspJhxZO2cEIDVhHr2M=s800-c-k-c0x00ffffff-no-rj',
      'width': 800,
      'height': 800}},
    'localized': {'title': 'Doug DeMuro', 'description': ''}},
   'statistics': {'viewCount':

In [49]:
type(channel_data['pageInfo']['totalResults'])

int

# Transform

##### Tranform channel stats

In [35]:
channel_stats ={
        'channel_id': channel_data['items'][0]['id'],
        'channel_title' : channel_data['items'][0]['snippet']['title'],
        'publishedAt' : channel_data['items'][0]['snippet']['publishedAt'],
        'subscribers' : channel_data['items'][0]['statistics']['subscriberCount'],
        'views' : channel_data['items'][0]['statistics']['viewCount'],
        'videos' : channel_data['items'][0]['statistics']['videoCount']
}

channel_stats_df = pd.DataFrame([channel_stats])

# change datatypes
channel_stats_df['views'] = channel_stats_df['views'].astype(int)
channel_stats_df['videos'] = channel_stats_df['videos'].astype(int)
channel_stats_df['subscribers'] = channel_stats_df['subscribers'].astype(int)
channel_stats_df['publishedAt'] = dt.datetime.strptime(channel_stats_df['publishedAt'].str.split('T')[0][0], '%Y-%m-%d').strftime('%Y-%m-%d')
channel_stats_df

,channel_id,channel_title,publishedAt,subscribers,views,videos
0,UCsqjHFMB_JYTaEnf_vmTNqg,Doug DeMuro,2013-09-11,4450000,1634723071,802


##### Transform video stats

In [36]:
video_data = get_video_stats(api_key)
video_data

KeyError: 'items'

In [ ]:
video_stats_df = pd.DataFrame(video_data)

#  change datatypes
video_stats_df['comments'] = video_stats_df['comments'].astype(int)
video_stats_df['likes'] = video_stats_df['likes'].astype(int)
video_stats_df['views'] = video_stats_df['views'].astype(int)
video_stats_df['publishedAt'] = pd.to_datetime(video_stats_df['publishedAt'])


# df.dtypes
video_stats_df

# Load

In [ ]:
def dump_to_csv():
    channel_stats_df.to_csv(f"results/channel_stats/{date.today().strftime('%Y-%m-%d')}.csv", index = False)
    video_stats_df.to_csv(f"results/video_stats/{date.today().strftime('%Y-%m-%d')}.csv", index = False)
    
dump_to_csv()

In [40]:
total = 323
total/50
x=0

while (x<(total/50)):
    print('hello')
    x += 1

hello
hello
hello
hello
hello
hello
hello


In [42]:
834/50

16.68